In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import os
import json
from dotenv import load_dotenv
from tqdm import tqdm_notebook as tqdm

# create a webdriver instance
# latest headless options: https://stackoverflow.com/questions/46753393/how-to-run-headless-firefox-with-selenium-in-python
options = Options()
options.add_argument("-headless")
options.add_argument("-privatebrowsing")

driver = webdriver.Firefox(options=options)
# print("Headless Firefox Initialized")

# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")
# driver.get("http://google.com/")
 
# waiting for the page to load
time.sleep(3)

In [2]:
load_dotenv()

True

Login To LinkedIn using provided credentials

In [3]:
# entering username
_USERNAME = driver.find_element(By.ID, "username")
 
# Enter Your Email Address
_USERNAME.send_keys(os.getenv('USERNAME'))  
 
# entering password
_PASSWORD = driver.find_element(By.ID, "password")
 
# Enter Your Password
_PASSWORD.send_keys(os.getenv('PASSWORD'))

# Clicking on the log in button
# Format (syntax) of writing XPath --> 
# //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

# waiting for the page to load
time.sleep(3)

Bypass Job search Inputs and use user provided base url

In [4]:
# driver.get("https://www.linkedin.com/jobs/search/")
# # jobs.click()

# wait = WebDriverWait(driver, 10)
# wait.until(EC.url_changes('https://www.linkedin.com/jobs/search/'))
# get_url = driver.current_url
# print("The current url is:"+str(get_url))

# # waiting for the page to load
# time.sleep(3)

In [5]:
# # entering username
# TITLE_SKILL_COMPANY_INPUT = driver.find_element(By.XPATH, "//*[@id='jobs-search-box-keyword-id-ember142']")
 
# # Enter Your Email Address
# TITLE_SKILL_COMPANY_INPUT.send_keys(os.getenv('TITLE_SKILL_COMPANY'))  
 
# # entering password
# LOCATION_INPUT = driver.find_element(By.XPATH, "//*[@id='jobs-search-box-location-id-ember142']")
 
# # Enter Your Password
# LOCATION_INPUT.send_keys(os.getenv('LOCATION'))

# # Clicking on the log in button
# # Format (syntax) of writing XPath --> 
# # //tagname[@attribute='value']
# driver.find_element(By.CLASS_NAME, "jobs-search-box__submit-button").click()

# # waiting for the page to load
# time.sleep(3)

Navigate to user provided url

In [6]:
driver.get(os.getenv("USER_PROVIDED_URL"))
time.sleep(3)

Save first page for analysis

In [7]:
# first_page = driver.page_source
 
# soup = BeautifulSoup(first_page, 'lxml')
# # print(soup)
# with open("first-page.html", "w", encoding='utf-8') as file:
#     file.write(str(soup))

Extract information from first page

In [8]:
# with open('first_page.html', 'r', encoding='utf-8') as f:
    # contents = f.read()

first_page = driver.page_source

soup = BeautifulSoup(first_page, 'lxml')
num_results_div = soup.find("div", attrs={"class": "jobs-search-results-list__subtitle"})
num_results_text = num_results_div.span.get_text(strip=True)
int_num_results = int(num_results_text.split()[0])
assert isinstance(int_num_results, int) and int_num_results > 0

int_num_results

463

In [9]:
job_post_ids=set()
output_json = "sample.json"
max_num = 0 # loose approximation for number of jobs to scrape

try:
  with open(output_json) as infile:
    job_desc_id = json.load(infile)

  print(f"Retrieved {len(job_desc_id)} jobs from save state.")

except OSError as e:
  print(f"Previous save state: {output_json} not found: {e}")
  job_desc_id = {}    # job_id: {job_title: x, job_desc: y}

Retrieved 100 jobs from save state.


Fetch all job post ids from first page

In [10]:
# _base_url = os.getenv("USER_PROVIDED_URL")
# # _postfix = "&start=25"
# # page_url = _base_url + _postfix

# driver.get(_base_url)
# time.sleep(3)

In [11]:
# # with open('first_page.html', 'r', encoding='utf-8') as f:

# #     contents = f.read()

# #     soup = BeautifulSoup(contents, 'lxml')
# #     job_post_divs = soup.find_all('div', attrs={"data-view-name": "job-card"})

# #     for div in job_post_divs:
# #         job_post_ids.append(div["data-job-id"])

# #     job_post_lis = soup.find_all('li', attrs={'data-occludable-job-id': True})

# #     for li in job_post_lis:
# #         job_post_ids.append(li["data-occludable-job-id"])


# page = driver.page_source
 
# soup = BeautifulSoup(page, 'lxml')
# job_post_divs = soup.find_all('div', attrs={"data-view-name": "job-card"})

# for div in job_post_divs:
#     job_post_ids.add(div["data-job-id"])

# job_post_lis = soup.find_all('li', attrs={'data-occludable-job-id': True})

# for li in job_post_lis:
#     job_post_ids.add(li["data-occludable-job-id"])

# print(len(job_post_ids), job_post_ids)

In [12]:
# calculate safety:
cal_safety=((461-25)//25)*25
if max_num == 0:
    num_to_scrape = cal_safety
else:
    if (0<max_num) and (max_num<cal_safety):
        num_to_scrape = max_num
    else:
        print("Invalid max_num supplied, using cal_safety")
        num_to_scrape = cal_safety

for i in range(0, num_to_scrape, 25):
    _base_url = os.getenv("USER_PROVIDED_URL")
    _postfix = f"&start={i}"
    page_url = _base_url + _postfix

    if i == 0:
        driver.get(_base_url)
    else:
        driver.get(page_url)
    time.sleep(3)

    page = driver.page_source
    
    soup = BeautifulSoup(page, 'lxml')
    job_post_divs = soup.find_all('div', attrs={"data-view-name": "job-card"})

    for div in job_post_divs:
        job_post_ids.add(div["data-job-id"])

    job_post_lis = soup.find_all('li', attrs={'data-occludable-job-id': True})

    for li in job_post_lis:
        job_post_ids.add(li["data-occludable-job-id"])

    print(len(job_post_ids))
    

25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425


In [13]:
# _base_url = os.getenv("USER_PROVIDED_URL")
# _postfix = "&start=25"
# page_url = _base_url + _postfix

# driver.get(page_url)
# time.sleep(3)

In [14]:
# # with open('first_page.html', 'r', encoding='utf-8') as f:

# #     contents = f.read()

# #     soup = BeautifulSoup(contents, 'lxml')
# #     job_post_divs = soup.find_all('div', attrs={"data-view-name": "job-card"})

# #     for div in job_post_divs:
# #         job_post_ids.append(div["data-job-id"])

# #     job_post_lis = soup.find_all('li', attrs={'data-occludable-job-id': True})

# #     for li in job_post_lis:
# #         job_post_ids.append(li["data-occludable-job-id"])

# ###

# page = driver.page_source
 
# soup = BeautifulSoup(page, 'lxml')
# job_post_divs = soup.find_all('div', attrs={"data-view-name": "job-card"})

# for div in job_post_divs:
#     job_post_ids.add(div["data-job-id"])

# job_post_lis = soup.find_all('li', attrs={'data-occludable-job-id': True})

# for li in job_post_lis:
#     job_post_ids.add(li["data-occludable-job-id"])

# print(len(job_post_ids), job_post_ids)

In [15]:
for id in tqdm(job_post_ids, desc='completed job_ids'):
    if id in job_desc_id:
        print(f"skipping job_id: {id}; duplicated/ found in save slate")
        continue

    driver.get(f"https://www.linkedin.com/jobs/search/?currentJobId={id}")
    time.sleep(3)

    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')

    job_title_element = soup.find('span', class_='job-details-jobs-unified-top-card__job-title-link')
    job_title = job_title_element.text.strip()

    job_post_divs = soup.find('div', attrs={"id": "job-details"})
    # get text
    job_post_div_texts = job_post_divs.get_text()
    # replace non-ascii with space
    p1_job_desc = ''.join([i if ord(i) < 128 else ' ' for i in job_post_div_texts])
    # convert all to lowercase
    p2_job_desc = p1_job_desc.lower()
    # p1_job_desc = job_post_divs.get_text().encode('ascii', errors='ignore').decode().lower()
    p3_job_desc = " ".join(p2_job_desc.split(r"\n")).replace("about the job", " ").replace("this job is sourced from a job board.", " ")
    # merge consecutive spaces
    p4_job_desc = " ".join(p3_job_desc.split())
    # merge consecutive spaces
    pf_job_desc = r"\n".join(p4_job_desc.split(r"\n"))

    job_desc_id[id] = {
        "job_title": job_title,
        "job_desc": pf_job_desc
    }

C:\Users\kelvin\AppData\Local\Temp\ipykernel_20572\2827516010.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm(job_post_ids, desc='completed job_ids'):


completed job_ids:   0%|          | 0/425 [00:00<?, ?it/s]

skipping job_id: 3771842589; duplicated/ found in save slate
skipping job_id: 3789945931; duplicated/ found in save slate
skipping job_id: 3791409552; duplicated/ found in save slate
skipping job_id: 3766022721; duplicated/ found in save slate
skipping job_id: 3794784458; duplicated/ found in save slate
skipping job_id: 3774303768; duplicated/ found in save slate
skipping job_id: 3669874647; duplicated/ found in save slate
skipping job_id: 3738246097; duplicated/ found in save slate
skipping job_id: 3788272820; duplicated/ found in save slate
skipping job_id: 3736318809; duplicated/ found in save slate
skipping job_id: 3787800069; duplicated/ found in save slate
skipping job_id: 3674404146; duplicated/ found in save slate
skipping job_id: 3745348244; duplicated/ found in save slate
skipping job_id: 3728589956; duplicated/ found in save slate
skipping job_id: 3775444318; duplicated/ found in save slate
skipping job_id: 3669875571; duplicated/ found in save slate
skipping job_id: 3784377

In [16]:
# job_desc_id

In [17]:
with open(output_json, "w") as outfile:
    json.dump(job_desc_id, outfile)

In [18]:
driver.close()